In [ ]:

# mean/box rating per month / yr agg by gender

# korelace poctu her a ratingu

In [1]:
import glob
import pandas as pd

data_files = glob.glob("chess_ratings/*.csv")
all_dfs = [pd.read_csv(file, sep='|') for file in data_files]

/tmp/ipykernel_147459/750563276.py:5: DtypeWarning: Columns (7) have mixed types.Specify dtype option on import or set low_memory=False.
  all_dfs = [pd.read_csv(file, sep='|') for file in data_files]


In [2]:
from spark_utils import preprocess_data

data = pd.concat(all_dfs)
data = preprocess_data(data)

In [3]:
from pyspark.sql import SparkSession

spark = SparkSession.builder \
    .master('local[*]') \
    .config("spark.driver.memory", "20g") \
    .appName('chess-app') \
    .getOrCreate()

spark.sparkContext.setLogLevel("ERROR")

Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
22/05/24 12:42:01 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [4]:
spark.conf.set("spark.sql.execution.arrow.pyspark.enabled", "true")

df = spark.createDataFrame(data)

In [5]:
seed = 42

In [6]:
import warnings
from experiments import create_dataset

data = create_dataset(df, n_months=6, last_year=2019, last_month=6)

In [7]:
import importlib
import model_utils
importlib.reload(model_utils)

<module 'model_utils' from '/opt/data/Gabi/code/ds_report/model_utils.py'>

In [8]:
from model_utils import make_models

models = make_models()
test_model = models['PCA_20-RF_100']

In [9]:
data_equal = create_dataset(df, n_months=6, last_year=2019, last_month=6, bins='equal')

In [10]:
from model_utils import get_data_split

data = get_data_split(data)
data_equal = get_data_split(data_equal)

In [13]:
from model_utils import eval_model_on_data

print(eval_model_on_data(data, test_model)['metrics'])
print(eval_model_on_data(data_equal, test_model)['metrics'])

{'auc': 0.7421994060034337, 'accuracy': 0.9024293254283855, 'cm': array([[22825,   668],
       [ 2031,  2138]])}
{'auc': 0.7433058378637296, 'accuracy': 0.9024654761044032, 'cm': array([[22815,   678],
       [ 2020,  2149]])}


In [12]:
import importlib
import model_utils
importlib.reload(model_utils)

<module 'model_utils' from '/opt/data/Gabi/code/ds_report/model_utils.py'>